# Circuit Tutorial - QuantumCircuitsMPS.jl

This tutorial demonstrates the Circuit API workflow for building, visualizing, and simulating quantum circuits in QuantumCircuitsMPS.jl. The Circuit API provides a lazy/symbolic representation that separates circuit construction from execution, enabling visualization and inspection before running expensive simulations.

## Key Concepts

1. **Build**: Circuit created with do-block syntax (lazy/symbolic)
2. **Visualize**: SVG diagrams without execution
3. **Simulate**: Deterministic execution with explicit RNG seeds

## Why Use the Circuit API?

- ✓ Inspect circuit structure before expensive simulation
- ✓ Reuse same circuit with different initial states
- ✓ Export circuit diagrams for documentation/papers
- ✓ Separate logical structure from execution details

## Setup

First, activate the project environment and import QuantumCircuitsMPS:

In [1]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using QuantumCircuitsMPS

  Activating project at `/mnt/d/Rutgers/QuantumCircuitsMPS.jl`


## Section 1: Setup and Parameters

Define the system parameters for our quantum circuit:

In [2]:
# Define system parameters
const L = 4                    # System size (number of qubits)
const bc = :periodic           # Boundary conditions (:periodic or :open)
const n_steps = 50             # Number of circuit timesteps
const p_reset = 0.3            # Probability of reset operation

println("=" ^ 70)
println("Circuit Tutorial - QuantumCircuitsMPS.jl")
println("=" ^ 70)
println()
println("Parameters:")
println("  L = $L (system size)")
println("  bc = $bc (boundary conditions)")
println("  n_steps = $n_steps (circuit timesteps)")
println("  p_reset = $p_reset (reset probability)")
println()

Circuit Tutorial - QuantumCircuitsMPS.jl

Parameters:
  L = 4 (system size)
  bc = periodic (boundary conditions)
  n_steps = 50 (circuit timesteps)
  p_reset = 0.3 (reset probability)



## Section 2: Building Circuits with Do-Block Syntax

The Circuit API uses a lazy/symbolic representation. When you build a circuit, no quantum operations are executed - instead, the circuit structure is recorded as a data structure that can be inspected, visualized, and executed later.

The do-block syntax provides a clean way to define circuit operations:
- `apply!(c, gate, geometry)` - for deterministic gates
- `apply_with_prob!(c; rng=:ctrl, outcomes=[...])` - for stochastic operations

In [3]:
println("Building quantum circuit...")
println()

# Build circuit with stochastic operations
# This circuit models a measurement-reset protocol:
#   - With probability p_reset: Reset qubit to |0⟩ state
#   - With probability 1-p_reset: Apply random unitary (HaarRandom)
#
# The StaircaseRight(1) geometry applies operations in a staircase pattern,
# moving one site to the right each timestep (with periodic wrapping)
L=4
p_reset = 0.
n_steps=10
circuit = Circuit(L=L, bc=bc, n_steps=n_steps) do c
    apply_with_prob!(c; rng=:ctrl, outcomes=[
        (probability=p_reset, gate=Reset(), geometry=StaircaseLeft(1)),
        (probability=1-p_reset, gate=HaarRandom(), geometry=StaircaseRight(1))
    ])
end

println("✓ Circuit built successfully")
println("  Total timesteps: $(circuit.n_steps)")
println("  System size: $(circuit.L) qubits")
println("  Boundary conditions: $(circuit.bc)")
println()

Building quantum circuit...

✓ Circuit built successfully
  Total timesteps: 10
  System size: 4 qubits
  Boundary conditions: periodic



In [4]:
p_reset

0.0

## Section 3: Adding Deterministic Gates

In addition to stochastic operations, you can add deterministic gates using `apply!()`. This example builds a circuit with both types of operations:

In [4]:
println("Building circuit with mixed gate types...")
println()

mixed_circuit = Circuit(L=L, bc=:open, n_steps=20) do c
    # Apply PauliX gate to site 1
    apply!(c, PauliX(), SingleSite(1))
    
    # Stochastic operation: probabilistic reset
    apply_with_prob!(c; rng=:ctrl, outcomes=[
        (probability=0.5, gate=Reset(), geometry=StaircaseLeft(1)),
        (probability=0.5, gate=HaarRandom(), geometry=StaircaseRight(1))
    ])
    
    # Apply PauliZ gate to last site
    apply!(c, PauliZ(), SingleSite(L))
end

println("✓ Mixed-gate circuit built")
println("  Operations include: PauliX, Reset, HaarRandom, PauliZ")
println()

Building circuit with mixed gate types...

✓ Mixed-gate circuit built
  Operations include: PauliX, Reset, HaarRandom, PauliZ


  Operations include: PauliX, Reset, HaarRandom, PauliZ



## Section 5: SVG Visualization (Optional)

The `plot_circuit` function generates publication-quality SVG diagrams. This requires Luxor.jl as a weak dependency (optional).

We use try-catch to gracefully handle the case where Luxor is not installed:

In [5]:
println("SVG Visualization (optional - requires Luxor.jl):")
println()

# Ensure output directory exists
mkpath("examples/output")

# Try loading Luxor for SVG export
try
    @eval using Luxor
    plot_circuit(circuit; seed=42, filename="examples/output/circuit_tutorial.svg")
    println("✓ SVG diagram saved to examples/output/circuit_tutorial.svg")
    println("  You can open this file in a web browser or include it in LaTeX documents")
catch e
    if isa(e, ArgumentError) && occursin("Luxor", string(e))
        println("ℹ Luxor.jl not available - SVG export skipped")
        println("  To enable SVG export, install Luxor: ]add Luxor")
    else
        println("⚠ SVG export failed: $(sprint(showerror, e))")
    end
end
println()

SVG Visualization (optional - requires Luxor.jl):

✓ SVG diagram saved to examples/output/circuit_tutorial.svg
  You can open this file in a web browser or include it in LaTeX documents


  You can open this file in a web browser or include it in LaTeX documents



In [7]:
short_circuit

UndefVarError: UndefVarError: `short_circuit` not defined

In [9]:
plot_circuit(mixed_circuit; seed=42, filename="examples/output/circuit_tutorial.svg")

true

In [14]:
BornProbability

BornProbability

In [10]:
list_observables()

2-element Vector{String}:
 "DomainWall"
 "BornProbability"

## Section 6: Observables and Simulating Circuits

Once a circuit is built and visualized, we can execute it using `simulate!()`. The simulation requires:

1. A SimulationState with initialized MPS and RNG registry
2. The circuit to execute
3. Number of trajectory samples (n_circuits)

**TIP**: Use `list_observables()` to discover all available observable types that can be measured during simulation (e.g., DomainWall, Entanglement, etc.)

**CRITICAL**: Using the same RNG seed for visualization and simulation ensures they show identical stochastic branch choices.

In [6]:
println("Simulating circuit (seed=42)...")
println()

# Create simulation state with RNG registry
# Each RNG source (ctrl, proj, haar, born) gets its own deterministic seed
state = SimulationState(
    L = L, 
    bc = bc, 
    rng = RNGRegistry(ctrl=42, proj=1, haar=2, born=3)
)

# Initialize state to product state with x=1/16
# This represents all qubits in a coherent state |ψ⟩ = |+⟩ ⊗ ... ⊗ |+⟩
# where |+⟩ = (|0⟩ + |1⟩)/√2, rotated by angle θ with cos²(θ/2) = 1/16
initialize!(state, ProductState(x0=1//16))

# Execute circuit with one trajectory
simulate!(circuit, state; n_circuits=1)

println("✓ Circuit simulation complete")
println("  Final state prepared with $(n_steps) timesteps")
println("  Stochastic branches sampled with seed=42")
println()

Simulating circuit (seed=42)...

✓ Circuit simulation complete
  Final state prepared with 10 timesteps
  Stochastic branches sampled with seed=42


  Final state prepared with 10 timesteps
  Stochastic branches sampled with seed=42



In [12]:
state.observables

Dict{Symbol, Vector}()

## Section 7: Comparing Visualization and Simulation

A key feature of the Circuit API is that visualization and simulation are deterministically linked via RNG seeds. The same seed produces:
- Same gate sequence in SVG visualization
- Same quantum trajectory in simulation

This ensures that what you see in the diagram is exactly what gets executed.

In [8]:
println("Verification: Deterministic consistency")
println()

# Run simulation with same seed
state_verify = SimulationState(
    L = L, 
    bc = bc, 
    rng = RNGRegistry(ctrl=42, proj=1, haar=2, born=3)
)
initialize!(state_verify, ProductState(x0=1//16))

# Track an observable to demonstrate observable access later
track!(state_verify, :dw1 => DomainWall(; order=1, i1_fn=() -> 1))
record!(state_verify)

simulate!(circuit, state_verify; n_circuits=1)

println("✓ Simulation with seed=42 complete")
println()
println("ℹ The visualization and simulation used identical RNG branches")
println("  → What you see in the diagram is what gets executed")
println("  → This enables reliable debugging and reproducible results")
println()

Verification: Deterministic consistency

✓ Simulation with seed=42 complete

ℹ The visualization and simulation used identical RNG branches
  → What you see in the diagram is what gets executed
  → This enables reliable debugging and reproducible results



ℹ The visualization and simulation used identical RNG branches
  → What you see in the diagram is what gets executed
  → This enables reliable debugging and reproducible results



## Demo A: Track at Every Circuit Execution

This demo shows how to record observables **after each circuit execution** using `record_every=1`. The Circuit API pattern with `simulate!()` executes multiple independent circuits and records measurements at specified intervals.

With `n_circuits=3` and `record_every=1`, we get 3 recordings (one after each circuit execution).

In [9]:
# Demo A: Track at every circuit execution
demo_circuit = Circuit(L=4, bc=:periodic, n_steps=1) do c
    apply!(c, HaarRandom(), StaircaseRight(1))
end

state_a = SimulationState(L=4, bc=:periodic, rng=RNGRegistry(ctrl=1, proj=2, haar=3, born=4))
initialize!(state_a, ProductState(x0=1//16))
track!(state_a, :dw => DomainWall(; order=1, i1_fn=() -> 1))

simulate!(demo_circuit, state_a; n_circuits=3, record_initial=false, record_every=1)
println("Every-circuit recordings (", length(state_a.observables[:dw]), " values): ", state_a.observables[:dw])

Every-gate recordings (3 values): [3.3542315770238003, 2.436067131437081, 3.6295551469908536]
[3.3542315770238003, 2.436067131437081, 3.6295551469908536]


In [11]:
state_a

SimulationState(ITensorMPS.MPS(4), ITensors.Index[(dim=2|id=576|"Qubit,Site,n=1"), (dim=2|id=462|"Qubit,Site,n=2"), (dim=2|id=173|"Qubit,Site,n=3"), (dim=2|id=270|"Qubit,Site,n=4")], [1, 3, 4, 2], [1, 4, 2, 3], 4, :periodic, 2, 1.0e-10, 100, RNGRegistry(Dict{Symbol, Random.AbstractRNG}(:proj => Random.MersenneTwister(2), :ctrl => Random.MersenneTwister(1), :haar => Random.MersenneTwister(3, (0, 1002, 0, 99)), :born => Random.MersenneTwister(4), :state_init => Random.MersenneTwister(0))), Dict{Symbol, Vector}(:dw => [3.3542315770238003, 2.436067131437081, 3.6295551469908536]), Dict{Symbol, Any}(:dw => DomainWall(1, var"#23#24"())))

## Demo B: Track with Sparse Recording

This demo shows how to use **sparse recording** with `record_every=3`. Instead of recording after every circuit, we only record at specific intervals.

With `n_circuits=3` and `record_every=3`, the recording formula `(circuit_idx - 1) % record_every == 0` triggers at circuit 1 (since (1-1)%3==0), and the final circuit always records, giving us 2 recordings total (circuits 1 and 3). This is more efficient for large-scale simulations where you only need snapshots at key points.

In [ ]:
# Demo B: Track with sparse recording (every 3rd circuit)
state_b = SimulationState(L=4, bc=:periodic, rng=RNGRegistry(ctrl=1, proj=2, haar=3, born=4))
initialize!(state_b, ProductState(x0=1//16))
track!(state_b, :dw => DomainWall(; order=1, i1_fn=() -> 1))

simulate!(demo_circuit, state_b; n_circuits=3, record_initial=false, record_every=3)
println("Sparse recordings (", length(state_b.observables[:dw]), " values - circuits 1 and 3): ", state_b.observables[:dw])

## Section 8: Accessing Recorded Observable Data

After tracking and recording observables during simulation, you can access the measured data through the `state.observables` dictionary using the observable name as the key.

In [17]:
println("Accessing recorded observable data:")
println()

# Access the domain wall observable we tracked
dw_values = state_verify.observables[:dw1]
println("Domain wall measurements: ", dw_values)
println()
println("ℹ Observable data is stored as a vector, with one measurement per timestep")
println("  You can plot, analyze, or export this data for further study")
println()

Accessing recorded observable data:

Domain wall measurements: Float64[]

ℹ Observable data is stored as a vector, with one measurement per timestep
  You can plot, analyze, or export this data for further study



## Summary

### What You Learned

1. Build circuits with do-block syntax (lazy/symbolic)
2. Visualize circuits with `plot_circuit` (SVG)
3. Simulate circuits with `simulate!()` (deterministic execution)
4. Use RNG seeds to ensure visualization matches simulation
5. Track and record observables during simulation
6. Access recorded observable data via `state.observables`

### Circuit API Workflow

**Build → Visualize → Inspect → Simulate → Analyze**

## Next Steps

Explore these possibilities:

- Experiment with different gate types (Reset, HaarRandom, PauliX, PauliZ)
- Try different geometries (SingleSite, StaircaseRight, StaircaseLeft)
- Vary system size L and timesteps n_steps
- Compare periodic vs open boundary conditions
- Install Luxor.jl for publication-quality SVG diagrams
- Explore different observables with `list_observables()`

---

**Tutorial complete!**